 # MPA-MLF, Lab 7 - Convolutional Neural Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


0. Import libraries

In [ ]:
from tensorflow import keras
from keras.models import Sequential
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from keras.datasets import cifar10
import zipfile
import os
import pandas as pd
from PIL import Image
from keras.utils import image_dataset_from_directory
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping


###################################
font = {'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [ ]:
X_train=[]
for image in os.listdir("/content/drive/MyDrive/data_projet/train/train_data_unlabeled"):
  im=Image.open(os.path.join("/content/drive/MyDrive/data_projet/train/train_data_unlabeled",image))
  im1=np.asarray(im)
  im.close()
  im1=im1[:,:,:3]
  X_train.append(im1)

X_train=np.asarray(X_train)


X_test=[]
for image in os.listdir("/content/drive/MyDrive/data_projet/test/test_data_unlabeled"):
  im=Image.open(os.path.join("/content/drive/MyDrive/data_projet/test/test_data_unlabeled",image))
  im1=np.asarray(im)
  im.close()
  im1=im1[:,:,:3]
  X_test.append(im1)

X_test=np.asarray(X_test)




In [ ]:
X_train_scaled = X_train.astype('float32') / 255.0

y_train=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/data_projet/y_train.csv"))
print(y_train)
y_train=np.asarray(y_train.drop("id", axis=1))

y_train_encoded = to_categorical(y_train, num_classes=4)

          id  target
0          0       0
1          1       1
2          2       0
3          3       2
4          4       2
...      ...     ...
16177  16177       1
16178  16178       3
16179  16179       2
16180  16180       0
16181  16181       1

[16182 rows x 2 columns]


In [ ]:
y_train

array([[0],
       [1],
       [0],
       ...,
       [2],
       [0],
       [1]])

### 1. Dataset

You can load the dataset using the following code

#### 1.2 Dataset examination

Using the following code, display random images,

#### 1.3 Dataset preprocessing

Perform the necessary data preprocessing. The best way to preprocess the data would be one hot encoding for the target variable and normalization for the input variable (using min-max or z-score normalization)

### 2. Build the model

#### 2.1 Define the model structure

In [ ]:

###################################
# Write your own code here #

from keras.layers import Input
from keras.layers import Dropout

input_shape = (45, 51, 3)
model = Sequential()
#Convolutional_layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# Maxpooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))


# Flatten Layer
model.add(Flatten())

# Dense Layer
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))


###################################


#### 2.2 Compile the model

In [ ]:

###################################
# Write your own code here #

optimizer = Adamax(learning_rate = 0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])



model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 43, 49, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 21, 24, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 22, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 9, 11, 64)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6336)              0         
                                                                 
 dense (Dense)               (None, 128)               8

### 3. Training stage

#### 3.1 Model training

In [ ]:
###################################
# Write your own code here #
callback = EarlyStopping(monitor='loss', patience=3)
history = model.fit(X_train_scaled, y_train_encoded, epochs=40, batch_size=150, validation_split = 0.2)#, callbacks=[callback])


###################################

Epoch 1/40
87/87 [==============================] - 50s 561ms/step - loss: 1.3397 - accuracy: 0.3281 - val_loss: 1.3349 - val_accuracy: 0.3457
Epoch 2/40
87/87 [==============================] - 53s 611ms/step - loss: 1.3365 - accuracy: 0.3320 - val_loss: 1.3339 - val_accuracy: 0.3457
Epoch 3/40
87/87 [==============================] - 47s 537ms/step - loss: 1.3363 - accuracy: 0.3320 - val_loss: 1.3335 - val_accuracy: 0.3457
Epoch 4/40
87/87 [==============================] - 49s 556ms/step - loss: 1.3362 - accuracy: 0.3320 - val_loss: 1.3330 - val_accuracy: 0.3457
Epoch 5/40
87/87 [==============================] - 48s 552ms/step - loss: 1.3361 - accuracy: 0.3320 - val_loss: 1.3334 - val_accuracy: 0.3457
Epoch 6/40
87/87 [==============================] - 46s 525ms/step - loss: 1.3359 - accuracy: 0.3320 - val_loss: 1.3329 - val_accuracy: 0.3457
Epoch 7/40
87/87 [==============================] - 46s 525ms/step - loss: 1.3357 - accuracy: 0.3320 - val_loss: 1.3326 - val_accuracy: 0.3457

#### 3.1 Model Evaluation on validation data

Plot the development of the training and validation loss, and training and validation metrics.

In [ ]:
###################################
import pandas as pd

# Prédictions du modèle sur les données de test
predictions = model.predict(X_test)

# Créer un DataFrame avec les colonnes 'id' et 'target'
df_predictions = pd.DataFrame({'id': range(len(predictions)), 'target': predictions.argmax(axis=1)})

# Enregistrer le DataFrame dans un fichier CSV avec les colonnes séparées
df_predictions.to_csv('predictions.csv', index=False, columns=['id', 'target'], sep=',')




150/150 [==============================] - 4s 27ms/step
